# Problem 1 – Binary Words and Operations  
**Author:** Michael Ferry  
**Date:** October 2025 

In [13]:
import numpy as np


## 32-bit Integers

In SHA-256 everything uses 32-bit numbers.  
Python doesn’t do that by default, so I made a small helper that turns any number into a 32-bit one.  
This makes sure the maths works the same way as in the standard.



I used the modulo (%) operator so the number always stays in the 32-bit range (0 to 4294967295).  
If it goes below 0 or above the limit, it wraps back around like a loop.  
This makes it act the same way as real 32-bit hardware does in SHA-256.

In [16]:
def to_uint32(x):

    return np.uint32(x % (1 << 32))

# Just runs a quick test to check the wrap
print(to_uint32(123))   # normal
print(to_uint32(-1))    # wraps around to 4294967295

123
4294967295
